# Manipulation.py: Shelf

In [2]:
import numpy as np
import robotic as ry
from time import sleep
import manipulation as manip
from shelf import generate_shelf

In [3]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.delFrame("panda_collCameraWrist")
C.getFrame("table").setShape(ry.ST.ssBox, size=[1., 1., .1, .02])

# Shelf
pos = np.array([-1., 0., .5])
generate_shelf(C, pos)

# Objects
for i in range(6):
	color = [0., 0., 0.]
	color[i%3] = 1.
	C.addFrame(f"box{i}") \
		.setPosition([(i%3)*.1, (i//3)*.1, .7]) \
		.setShape(ry.ST.ssBox, size=[.04, .04, .12, 0.005]) \
		.setColor(color) \
		.setContact(1) \
		.setMass(.1)
	
# for convenience, a few definitions:
gripper = "l_gripper"
palm = "l_palm"
table = "table"
box = "box0"

C.view()

0

In [4]:
def pick_place(boxes: list[str], shelf_coor: list[list[int]], vis: bool=False) -> bool:
	M = manip.ManipulationModelling(C, "shelf_task", helpers=[gripper])
	M.setup_pick_and_place_sequence(gripper, boxes, accumulated_collisions=False)

	for i, sc in enumerate(shelf_coor):
		time = i*2+1
		M.grasp_box(time, gripper, boxes[i], "l_palm", "x")
		M.komo.addObjective([time+1], ry.FS.insideBox, [f"small_box_inside_0_{sc[0]}_{sc[1]}", boxes[i]], ry.OT.ineq, [1e1])
		M.komo.addObjective([time+1], ry.FS.scalarProductXZ, [table, gripper], ry.OT.eq, [1e1], [1.])
	
	keypoints = M.solve()
	M.komo.report(plotOverTime=True)
	if not M.feasible:
		return False, []
	
	for i, k in enumerate(keypoints):
		C.setJointState(k)
		if i%2:
			C.attach(table, boxes[i//2])
		else:
			C.attach(gripper, boxes[i//2])
		C.view(False, f'step {i}')
		sleep(1.)

	return True, keypoints

	M1 = M.sub_motion(0)
	M1.keep_distance([.3,.7], "l_palm", obj, margin=.05)
	M1.retract([.0, .2], gripper)
	M1.approach([.8, 1.], gripper)
	path1 = M1.solve()
	if not M1.feasible:
		return False, []

	M2 = M.sub_motion(1)
	M2.keep_distance([], table, "panda_collCameraWrist")
	M2.keep_distance([.2, .8], table, obj, .04)
	M2.keep_distance([], "l_palm", obj)
	path2 = M2.solve()
	if not M2.feasible:
		return False, []

	if vis:
		M1.play(C, 1.)
		C.attach(gripper, obj)
		M2.play(C, 1.)
		C.attach(table, obj)

	path = np.append(path1, path2, 0)
	return True, path

In [5]:
boxes = [f"box{i}" for i in range(6)]
shelf_coor = [[2, 3], [2, 2], [1, 3], [1, 2], [1, 1], [3, 3]]
for i, sc in enumerate(shelf_coor):
	c = [0., 0., 0., .5]
	c[i%3] = 1.
	C.getFrame(f"small_box_inside_0_{sc[0]}_{sc[1]}") \
			.setColor(c)
C.view()
success, _ = pick_place(boxes, shelf_coor, vis=True)

  -- infeasible:shelf_task
     { time: 0.697593, evals: 200, done: 1, feasible: 0, sos: 0.430689, f: 0, ineq: 0.187772, eq: 0.725134 }
